In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
#Ejemplo para hacer scrapping con Selenium
init_time = time.time()

driver = web_driver()

while True:
    driver.get('https://www.goodreads.com/book/show/2767052-the-hunger-games')
    time.sleep(1)
    soup = bs(driver.page_source, 'html.parser')    #print(soup.prettify())
    if str(soup.text).find('Rate this book') > 1:
        break
    time.sleep(2)
   
driver.quit()
print(time.time()-init_time)

In [ ]:
#Ejemplo para hacer scrapping con requests
init_time = time.time()
html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text

while True:
    soup = bs(html)
    #print(soup.prettify())
    if str(soup.text).find('Rate this book') > 1:
        break
    time.sleep(2)
    html = requests.get("https://www.goodreads.com/book/show/2767052-the-hunger-games").text
  
print(time.time()-init_time)